### Preparations

In [58]:
pip install scrapingant-client

In [59]:
from bs4 import BeautifulSoup
from scrapingant_client import ScrapingAntClient
import pprint
import pandas as pd

In [60]:
total_results = []
search_results = []

In [61]:
scraping_token = '3c886402993c4c808448448877f7ed9e'

### Shopee

In [62]:
def get_shopee_result(query):
  processed_query = query.replace(' ', '%20').replace('&', '%26')
  url = f"https://shopee.co.id/search?keyword={processed_query}&ratingFilter=5"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('div', {"class": "shopee-search-item-result__item"})

  extracted_contents = []

  for content in contents:
    try:
      if len(content.find_all("img")) < 1:
        break
      extracted_content = {
        'image': "",
        'title': content.find("div", {"class", "yQmmFK _1POlWt _36CEnF"}).text,
        'url': f"https://shopee.co.id{content.find('a')['href']}",
        'price': int(content.find('span', {"class", "_29R_un"}).text.replace('.','')),
        'source': 'shopee',
        'rating': float(5.0),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue
  
  return extracted_contents

# pprint.pprint(get_shopee_result('mixer'))

### Tokopedia

In [63]:
def get_tokopedia_result(query):
  processed_query = query.replace(' ', '%20').replace('&', '%26')
  url = f"https://www.tokopedia.com/search?st=product&q={processed_query}&navsource=home"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('div', {"class": "css-12sieg3"})

  extracted_contents = []

  image_boilerplate = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHgAAAB4CAQAAACTbf5ZAAAAiklEQVR42u3PMQEAAAwCoNm/9Cr4Cw3IjYmwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsLCwsHDrASUpAHnJhbktAAAAAElFTkSuQmCC'

  for content in contents:
    try:
      if len(content.find_all("img")) < 1:
        break
      if 'ta.tokopedia.com/promo/v1' in content.find('a')['href']:
        continue
      extracted_content = {
        'image': content.find_all("img")[0]['src'] if len(content.find_all("img")) >= 1 and content.find_all("img")[0]['src'] != image_boilerplate else "",
        'title': content.find("div", {"class", "css-18c4yhp"}).text,
        'url': content.find('a')['href'],
        'price': int(content.find('div', {"class", "css-rhd610"}).text.split('Rp')[1].replace('.','').split(' ')[0]),
        'source': 'tokopedia',
        'rating': float(content.find('span', {"class", "css-etd83i"}).text if content.find('span', {"class", "css-etd83i"}).text != None else 0),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue

  return extracted_contents

# pprint.pprint(get_tokopedia_result('mixer'))

### Blibli

In [64]:
def get_blibli_result(query):
  processed_query = query.replace(' ', '%20').replace('&', '%26')
  url = f"https://www.blibli.com/cari/{processed_query}"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('div', {"class": "product__card product__card__five"})

  extracted_contents = []

  image_boilerplate = 'https://www.static-src.com/frontend/catalog/static/img/img-placeholder.76652f6.png'

  for content in contents:
    try:
      if len(content.find_all("img")) < 1:
        break
      extracted_content = {
        'image': content.find_all("img")[1]['src'] if len(content.find_all("img")) >= 1 and content.find_all("img")[1]['src'] != image_boilerplate else "",
        'title': content.find("div", {"class", "product__title"})['title'],
        'url': f"https://www.blibli.com{content.find('a')['href']}",
        'price': int(content.find('strong', {"class", "product__body__price__display"}).text.split('Rp')[1].split('\n')[0].replace('.','')),
        'source': 'blibli',
        'rating': float(content.find('span', {"class", "product__body__rating__stars__rating"}).text if content.find('span', {"class", "product__body__rating__stars__rating"}).text != None else 0),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue

  return extracted_contents

# pprint.pprint(get_blibli_result('mixer'))

### JD.ID

In [65]:
def get_jdid_result(query):
  processed_query = query.replace(' ', '%20').replace('&', '%26')
  url = f"https://www.jd.id/search?keywords={processed_query}"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('a', {"class": "sku-info"})

  extracted_contents = []

  image_boilerplate = 'st.3.cn/ept/1.0.0/ui/css/i/blank.gif'

  for content in contents:
    try:
      if len(content.find_all("img")) < 1:
        break
      extracted_content = {
        'image': content.find_all("img")[0]['src'].split('//')[1] if len(content.find_all("img")) >= 1 and content.find_all("img")[0]['src'].split('//')[1] != image_boilerplate else "",
        'title': content.find("div", {"class", "p-name"})['title'],
        'url': f"https://{content['href'].split('//')[1]}",
        'price': int(content.find('div', {"class", "p-price-zj"}).text.split('Rp ')[1].replace(',','')),
        'source': 'jdid',
        'rating': float(len(content.find_all('i', {"class", "rate-star rate-star-full"}))),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue
  
  return extracted_contents

# pprint.pprint(get_jdid_result('mixer'))

### Lazada

In [66]:
def get_lazada_result(query):
  processed_query = query.replace(' ', '+').replace('&', '%26')
  url = f"https://www.lazada.co.id/catalog/?q={processed_query}"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('div', {"class": "c3e8SH"})
  
  extracted_contents = []

  for content in contents:
    try:
      extracted_content = {
        'image': content.find_all("img")[0]['src'] if len(content.find_all("img")) >= 1 else "",
        'title': content.find("div", {"class", "c16H9d"}).find("a")['title'],
        'url': 'https:' + content.find("div", {"class", "c16H9d"}).find("a")['href'],
        'price': int(content.find("span", {"class", "c13VH6"}).text.split('Rp')[1].split('\n')[0].replace('.','')),
        'source': 'bukalapak',
        'rating': float(len(content.find_all('i', {"class", "c3dn4k c3EEAg"}))),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue

  return extracted_contents

# pprint.pprint(get_lazada_result('mixer'))

### Bukalapak

In [67]:
def get_bukalapak_result(query):
  processed_query = query.replace(' ', '%20').replace('&', '%26')
  url = f"https://www.bukalapak.com/products?search%5Bkeywords%5D={processed_query}"
  client = ScrapingAntClient(token=scraping_token)

  try:
    page_content = client.general_request(url).content
  except:
    return

  soup = BeautifulSoup(page_content)

  contents = soup.find_all('div', {"class": "bl-product-card__wrapper"})
  
  extracted_contents = []

  for content in contents:
    try:
      extracted_content = {
        'image': content.find_all("img")[0]['src'] if len(content.find_all("img")) >= 1 else "",
        'title': content.find("div", {"class", "bl-product-card__description-name"}).find("p").find("a").text,
        'url': content.find("div", {"class", "bl-product-card__description-name"}).find("p").find("a")['href'],
        'price': int(content.find("div", {"class", "bl-product-card__description-price"}).find("p").text.split('Rp')[1].split('\n')[0].replace('.','')),
        'source': 'bukalapak',
        'rating': float(content.find('div', {"class", "bl-product-card__description-rating"}).text if content.find('div', {"class", "bl-product-card__description-rating"}).text != None else 0),
        'category': query
      }
      extracted_contents.append(extracted_content)
      search_results.append(extracted_content)
    except:
      continue

  return extracted_contents

# pprint.pprint(get_bukalapak_result('mixer'))

### Multithread Scrapper

In [68]:
import numpy as np
import time
import threading
import math

def scrape_topic(query):
  start_time = time.time()
  search_results.clear()
  thread_list = []

  thread_list.append(threading.Thread(target=get_tokopedia_result, args=(query,)))
  thread_list.append(threading.Thread(target=get_shopee_result, args=(query,)))
  thread_list.append(threading.Thread(target=get_blibli_result, args=(query,)))
  thread_list.append(threading.Thread(target=get_jdid_result, args=(query,)))
  thread_list.append(threading.Thread(target=get_lazada_result, args=(query,)))
  thread_list.append(threading.Thread(target=get_bukalapak_result, args=(query,)))

  for thread in thread_list:
      thread.start()

  for thread in thread_list:
      thread.join()

  end_time = time.time()

  total_results.extend(search_results)

  return f"Topic: '{query}' -- Result: {len(search_results)} items -- Time taken: {str(end_time - start_time)} sec"

### Execute Web Scraping

In [69]:
list_of_topics = [
                  'Alat Pemotong Serbaguna',
                  'Capit Makanan',
                  'Celemek',
                  'Chopper',
                  'Grinder',
                  'Gunting Dapur',
                  'Korek Kompor',
                  'Parutan',
                  'Peeler',
                  'Pelindung Tangan',
                  'Pengasah Pisau',
                  'Pisau Dapur',
                  'Pisau Set',
                  'Talenan',
                  'Tatakan Gas',
                  'Termometer Makanan &',
                  'Minuman',
                  'Timer Masak',
                  'Coffee & Tea Maker',
                  'Cotton Candy Maker',
                  'Donut Maker',
                  'Ice Cream & Yoghurt Maker',
                  'Botol Minum',
                  'Cetakan Bento',
                  'Cup Bento',
                  'Kotak Makan',
                  'Lunch Box Set',
                  'Rantang',
                  'Tas Bekal',
                  'Tas Botol',
                  'Termos Air',
                  'Blender',
                  'Juicer',
                  'Kompor Listrik',
                  'Kulkas',
                  'Microwave',
                  'Mixer',
                  'Oven',
                  'Rice Cooker',
                  'Slow Cooker',
                  'Toaster',
                  'Alumunium Foil',
                  'Box Telur',
                  'Cooler Box',
                  'Food Display',
                  'Food Warmer',
                  'Ice Bucket',
                  'Plastic Wrap',
                  'Sealer Makanan',
                  'Tempat Buah & Sayur',
                  'Tempat Bumbu',
                  'Tempat Roti',
                  'Alat Pembuka Botol',
                  'Alat Pembuka Kaleng',
                  'Dispenser Air',
                  'Pompa Galon',
                  'Rak Dapur',
                  'Rak Piring',
                  'Regulator & Penghemat Gas',
                  'Sarung Galon',
                  'Sarung Kulkas',
                  'Timbangan Dapur',
                  'Water Purifier',
                  'Cangkir',
                  'Gelas & Mug',
                  'Gelas Wine',
                  'Nampan',
                  'Peralatan Makan Set',
                  'Peralatan Minum Set',
                  'Piring Makan',
                  'Piring & Mangkok Saji',
                  'Pitcher Minuman',
                  'Sedotan',
                  'Sendok & Garpu Makan',
                  'Cetakan Es & Pudding',
                  'Deep Fryer',
                  'Gelas Takar',
                  'Gilingan Daging',
                  'Griller',
                  'Kompor',
                  'Panci',
                  'Saringan Masak',
                  'Sendok Takar',
                  'Spatula & Sutil',
                  'Steamer',
                  'Dish Dryer',
                  'Sabut',
                  'Saringan Cuci Piring',
                  'Sikat Cuci Botol',
                  'Sponge Cuci Piring',
                  'Kursi Bar',
                  'Kursi Makan Outdoor',
                  'Kursi Ruang Makan',
                  'Meja Bar',
                  'Meja Makan Outdoor',
                  'Meja Ruang Makan'
                  ]

for index, topic in enumerate(list_of_topics, start=1):
  log_message = scrape_topic(topic)
  print(f"[{index}/{len(list_of_topics)}] {log_message}")

[1/96] Topic: 'Alat Pemotong Serbaguna' -- Result: 138 items -- Time taken: 27.137378215789795 sec
[2/96] Topic: 'Capit Makanan' -- Result: 165 items -- Time taken: 26.931535720825195 sec
[3/96] Topic: 'Celemek' -- Result: 144 items -- Time taken: 26.90737748146057 sec
[4/96] Topic: 'Chopper' -- Result: 143 items -- Time taken: 27.597044706344604 sec
[5/96] Topic: 'Grinder' -- Result: 191 items -- Time taken: 27.621601343154907 sec
[6/96] Topic: 'Gunting Dapur' -- Result: 170 items -- Time taken: 26.46969699859619 sec
[7/96] Topic: 'Korek Kompor' -- Result: 141 items -- Time taken: 27.385072231292725 sec
[8/96] Topic: 'Parutan' -- Result: 143 items -- Time taken: 18.816741466522217 sec
[9/96] Topic: 'Peeler' -- Result: 151 items -- Time taken: 26.825501918792725 sec
[10/96] Topic: 'Pelindung Tangan' -- Result: 136 items -- Time taken: 27.0272798538208 sec
[11/96] Topic: 'Pengasah Pisau' -- Result: 142 items -- Time taken: 27.337297201156616 sec
[12/96] Topic: 'Pisau Dapur' -- Result: 1

### Clean Final Data

In [70]:
import pandas as pd

df = pd.DataFrame.from_records(total_results)

df['title'] = df['title'].replace({'\n ': ''}, regex=True)
df['title'] = df['title'].replace({'  ': ''}, regex=True)

df.loc[df.source != 'tokopedia', 'image'] = ''

### Export to csv

In [71]:
df.to_csv('indonesia_marketplace_kitchenware.csv', index=False)

df

,image,title,url,price,source,rating,category
0,https://images.tokopedia.net/img/cache/200-squ...,Stein Cookware Diamond Knifes Stein / Pisau Se...,https://www.tokopedia.com/belilagicoid/stein-c...,259000,tokopedia,5.0,Alat Pemotong Serbaguna
1,https://images.tokopedia.net/img/cache/200-squ...,Freemir Alat Pemotong Sayur Multifungsi Nicer ...,https://www.tokopedia.com/freemirofficial/free...,69900,tokopedia,4.8,Alat Pemotong Serbaguna
2,https://images.tokopedia.net/img/cache/200-squ...,ALAT PEMOTONG SAYURAN CUTTER GRATER 12 IN 1 SE...,https://www.tokopedia.com/orinocosmart/alat-pe...,51868,tokopedia,4.8,Alat Pemotong Serbaguna
3,https://images.tokopedia.net/img/cache/200-squ...,BOLDe SUPER CUT GRANDE 5 IN 1 [5 BLADES] FREE ...,https://www.tokopedia.com/verry-lie/bolde-supe...,72871,tokopedia,4.9,Alat Pemotong Serbaguna
4,https://images.tokopedia.net/img/cache/200-squ...,ALAT PEMOTONG KENTANG SAYUR/ PARUTAN / PEELER ...,https://www.tokopedia.com/limanjaya88/alat-pem...,38500,tokopedia,4.8,Alat Pemotong Serbaguna
...,...,...,...,...,...,...,...
13585,,Taplak Meja Atau Table Runner Bahan Kain Kanva...,https://www.lazada.co.id/products/taplak-meja-...,37999,bukalapak,4.0,Meja Ruang Makan
13586,,TAPLAK MEJA PERCANTIK MEJA RUANG TAMU MEJA MAK...,https://www.lazada.co.id/products/taplak-meja-...,6000,bukalapak,5.0,Meja Ruang Makan
13587,,(BISA COD) TAPLAK MEJA PERSEGI PANJANG RUANG T...,https://www.lazada.co.id/products/bisa-cod-tap...,30667,bukalapak,2.0,Meja Ruang Makan
13588,,MEJA LIPAT KAKI BESI 120 X 60 DIONA AVEDA,https://www.lazada.co.id/products/meja-lipat-k...,420000,bukalapak,5.0,Meja Ruang Makan
